In [19]:
import polars as pl 
import polars_ds as pds
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS
# To get the english dictionary
import nltk
from nltk.corpus import words,stopwords
from nltk.stem import WordNetLemmatizer
import os 
import string
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer 

import warnings
warnings.filterwarnings('ignore')

In [6]:
try:
    nlp = spacy.load("en_core_web_sm")
except:
    print("Downloading spaCy NLP model...")
    print("This may take a few minutes and it's one time process...")
    os.system(
        "pip install https://huggingface.co/spacy/en_core_web_sm/resolve/main/en_core_web_sm-any-py3-none-any.whl")
    nlp = spacy.load("en_core_web_sm")

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Yafee
[nltk_data]     Ishraq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Read the data 

In [8]:
data = pl.read_csv('train.csv')

In [81]:
data.head()

id,keyword,location,text,target,cleaned_location,cleaned_text,cleaned_keyword
i64,str,str,str,i64,str,str,str
1,null,null,"""#earthquake deeds forgive alla…",1,null,"""earthquake deeds forgive allah…",null
4,null,null,"""forest fire near la ronge sask…",1,null,"""forest fire near la ronge sask…",null
5,null,null,"""officers. residents asked plac…",1,null,"""officers residents asked place…",null
6,null,null,"""13,000 people receive #wildfir…",1,null,"""people receive wildfires evacu…",null
7,null,null,"""smoke got sent school photo #w…",1,null,"""smoke got sent school photo wi…",null


# Null values 

There are very few null values in keyword column. Location contains 2533 null values. Later I will be looking at a word cloud to see if there are any patterns in the text columns.

In [9]:
data.null_count()

id,keyword,location,text,target
u32,u32,u32,u32,u32
0,61,2533,0,0


# Text cleaning

## Simple cleaning operations

In [10]:
# Remove leading and trailing characters
data = data.with_columns(pl.col(pl.String).str.strip_chars())

# To lowercase
data = data.with_columns(pl.col(pl.String).str.to_lowercase())

# Remove stopwords. Stopwords from the nltk library
stop_words = stopwords.words('english')

# Making expressions for all string columns. The pl.all() method is not returning the expected outp
stop_word_expression = [pl.col(c).str.split(" ").list.set_difference(stop_words).list.join(" ").name.keep() for c in data.select(pl.col(pl.String)).columns]

# Apply expressions
data = data.with_columns(*stop_word_expression)


## Defining regex

In [11]:
import re

# Regex for removing http url 
regex_http_url = r'http[s]?://\S+'

# Regex for punctuations
regex_pattern_punctuations = '[' + re.escape(string.punctuation) + ']'

# Regex for special characters
regex_pattern_special_characters = r'^[^A-Za-z0-9]*$'

# Regex for numbers
regex_pattern_numbers = r'\d'

# Remove ascii
regex_pattern_ascii = r'[^\p{Ascii}]'

# Regex for 2 or more whitespace
regex_whitespace = r'\s{2,}'


## Run text cleaning

In [12]:

data = data.with_columns(pl.col('location')
                  .str.replace_all(regex_http_url," ")
                  .str.replace_all(regex_pattern_punctuations," ")
                  .str.replace_all(regex_pattern_special_characters," ")
                  .str.replace_all(regex_pattern_numbers," ")
                  .str.replace_all(regex_pattern_ascii," ")
                  .str.replace_all(regex_whitespace," ")
                  .str.strip_chars()
                  .alias('cleaned_location'),

                  pl.col('text')
                  .str.replace_all(regex_http_url," ") 
                  .str.replace_all(regex_pattern_punctuations," ")
                  .str.replace_all(regex_pattern_special_characters," ")
                  .str.replace_all(regex_pattern_numbers," ")
                  .str.replace_all(regex_pattern_ascii," ")
                  .str.replace_all(regex_whitespace," ")
                  .str.strip_chars()
                  .alias('cleaned_text'),

                  pl.col('keyword')
                  .str.replace_all(regex_http_url," ")
                  .str.replace_all(regex_pattern_punctuations," ")
                  .str.replace_all(regex_pattern_special_characters," ")
                  .str.replace_all(regex_pattern_numbers," ")
                  .str.replace_all(regex_pattern_ascii," ")
                  .str.replace_all(regex_whitespace," ")
                  .str.strip_chars()
                  .alias('cleaned_keyword')
)

## Lemmatizaion

In [21]:
# Initialize
lemmatizer = WordNetLemmatizer()
 
print("rocks :", lemmatizer.lemmatize("rocks"))
print("corpora :", lemmatizer.lemmatize("corpora"))

rocks : rock
corpora : corpus


In [80]:
(data
.with_row_index(offset=1,name="row_id")
.select('row_id',pl.col('cleaned_text').str.split(" "))
.explode('cleaned_text')
.with_columns(pl.col('cleaned_text').map_elements(lambda x:lemmatizer.lemmatize(x)).suffix("_lemmatized"))
.filter(pl.col('cleaned_text')!=pl.col('cleaned_text_lemmatized'))
.filter(pl.col('cleaned_text_lemmatized').str.len_bytes()==1)
)

row_id,cleaned_text,cleaned_text_lemmatized
u32,str,str
1,"""us""","""u"""
43,"""us""","""u"""
70,"""us""","""u"""
85,"""us""","""u"""
103,"""ps""","""p"""
…,…,…
7233,"""us""","""u"""
7268,"""us""","""u"""
7311,"""us""","""u"""


[]

## TF-IDF

In [13]:
# Initialize 
tfidf_vectorizer = TfidfVectorizer()
tfidf_vector = tfidf_vectorizer.fit_transform(data['cleaned_text'])

In [14]:
# To polars
tf_idf_df = pl.from_numpy(tfidf_vector.toarray(),schema=list(tfidf_vectorizer.get_feature_names_out()))

# Add target variable 
tf_idf_df = tf_idf_df.hstack(data.select(pl.col('target').alias('target_ml')))


In [15]:
X = tf_idf_df.select(pl.all().exclude('target_ml'))
y = tf_idf_df.select('target_ml')

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [48]:
rf = RandomForestClassifier(n_estimators = 100)  
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

c:\Users\Yafee Ishraq\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier()

In [52]:
from sklearn import metrics 
print("ACCURACY OF THE MODEL:", metrics.accuracy_score(y_test, y_pred))

ACCURACY OF THE MODEL: 0.7655942219304005


In [57]:
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)

In [58]:
print("ACCURACY OF THE MODEL:", metrics.accuracy_score(y_test, y_pred))

ACCURACY OF THE MODEL: 0.7701904136572554
